In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install transformers datasets sacrebleu sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.6 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset, Dataset

dataset = load_dataset("juancavallotti/multilingual-gec")

print("datasetdict looks like:",dataset)

# Keep only relevant columns for eng
train_dataset = dataset["train"].filter(lambda example: example["lang"] == "en")
train_dataset = train_dataset.remove_columns(["transformation", "sec_transformation", "__index_level_0__"])

print(train_dataset[0:5])

README.md:   0%|          | 0.00/2.98k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/31.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/332k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/216318 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2186 [00:00<?, ? examples/s]

datasetdict looks like: DatasetDict({
    train: Dataset({
        features: ['lang', 'sentence', 'modified', 'transformation', 'sec_transformation', '__index_level_0__'],
        num_rows: 216318
    })
    test: Dataset({
        features: ['lang', 'sentence', 'modified', 'transformation', 'sec_transformation', '__index_level_0__'],
        num_rows: 2186
    })
})


Filter:   0%|          | 0/216318 [00:00<?, ? examples/s]

{'lang': ['en', 'en', 'en', 'en', 'en'], 'sentence': ['Plants, obviously, cannot move after they have put down roots.', 'I looked at the schedule.', "It's very hard to get rid of bad habits.", "Anyway, I think I've said enough.", 'Technologies allow you to do more things.'], 'modified': ["fix grammar: Plants, obviously, cannot moved after they hadn't put down roots.", 'fix grammar: I looked at schedule.', 'fix grammar: It am very hard to get rid of bad habits.', "fix grammar: Anyway, think I've said enough.", 'fix grammar: Technologies allow you to do most things.']}


In [5]:
processed_data = {
    "input_text": train_dataset["modified"],
    "target_text": train_dataset["sentence"]
}


# Convert to Hugging Face Dataset format
formatted_dataset = Dataset.from_dict(processed_data)


for i in range(5):  
    print(f"Example {i+1}:")
    print(f"  Input: {formatted_dataset['input_text'][i]}")
    print(f"  Target: {formatted_dataset['target_text'][i]}")
    print("-" * 50)

Example 1:
  Input: fix grammar: Plants, obviously, cannot moved after they hadn't put down roots.
  Target: Plants, obviously, cannot move after they have put down roots.
--------------------------------------------------
Example 2:
  Input: fix grammar: I looked at schedule.
  Target: I looked at the schedule.
--------------------------------------------------
Example 3:
  Input: fix grammar: It am very hard to get rid of bad habits.
  Target: It's very hard to get rid of bad habits.
--------------------------------------------------
Example 4:
  Input: fix grammar: Anyway, think I've said enough.
  Target: Anyway, I think I've said enough.
--------------------------------------------------
Example 5:
  Input: fix grammar: Technologies allow you to do most things.
  Target: Technologies allow you to do more things.
--------------------------------------------------


In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

In [7]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Tokenize input and target text
def tokenize_function(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=128)
    
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"]
    }


tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)

print(tokenized_dataset[0])

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/50862 [00:00<?, ? examples/s]

{'input_text': "fix grammar: Plants, obviously, cannot moved after they hadn't put down roots.", 'target_text': 'Plants, obviously, cannot move after they have put down roots.', 'input_ids': [2210, 19519, 10, 6041, 7, 6, 6865, 6, 1178, 2301, 227, 79, 12381, 31, 17, 474, 323, 8523, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
from sklearn.model_selection import train_test_split

train_size = int(0.8 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(train_size))
eval_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))

In [9]:
from transformers import  TrainerCallback

model = T5ForConditionalGeneration.from_pretrained("t5-base")

training_args = TrainingArguments(
    output_dir="/kaggle/working/t5-grammar-corrector",  
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_steps=50, 
    save_steps=500,  
    evaluation_strategy="steps",  
    eval_steps=500,  
    save_total_limit=2, 
    fp16=True,
    report_to="none", 
)


class ProgressCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"Step {state.global_step}: Loss = {logs.get('loss', 'N/A')}, LR = {logs.get('learning_rate', 'N/A')}", flush=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[ProgressCallback()],
)


trainer.train()

trainer.save_model("/kaggle/working/t5-grammar-corrector")

model.save_pretrained("/kaggle/working/t5-grammar-corrector")


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.027000,0.018151
1000,0.017800,0.015229
1500,0.017200,0.013195
2000,0.015400,0.012304
2500,0.014000,0.011244
3000,0.012400,0.011037
3500,0.011100,0.010668
4000,0.010700,0.010229
4500,0.009500,0.010147
5000,0.011500,0.009619


Step 50: Loss = 3.093, LR = 4.9672431865828094e-05
Step 100: Loss = 0.1017, LR = 4.9344863731656185e-05
Step 150: Loss = 0.043, LR = 4.9017295597484283e-05
Step 200: Loss = 0.0337, LR = 4.8689727463312375e-05
Step 250: Loss = 0.0284, LR = 4.8362159329140466e-05
Step 300: Loss = 0.0292, LR = 4.803459119496855e-05
Step 350: Loss = 0.0258, LR = 4.770702306079665e-05
Step 400: Loss = 0.0287, LR = 4.737945492662474e-05
Step 450: Loss = 0.0257, LR = 4.705188679245283e-05
Step 500: Loss = 0.027, LR = 4.672431865828092e-05
Step 500: Loss = N/A, LR = N/A


...

Step 6550: Loss = 0.0074, LR = 7.088574423480083e-06
Step 6600: Loss = 0.0082, LR = 6.761006289308176e-06
Step 6650: Loss = 0.0077, LR = 6.433438155136269e-06
Step 6700: Loss = 0.0076, LR = 6.105870020964361e-06
Step 6750: Loss = 0.0087, LR = 5.778301886792453e-06
Step 6800: Loss = 0.0082, LR = 5.4507337526205454e-06
Step 6850: Loss = 0.008, LR = 5.123165618448638e-06
Step 6900: Loss = 0.0087, LR = 4.79559748427673e-06
Step 6950: Loss = 0.0076, LR = 4.468029350104822e-06
Step 7000: Loss = 0.0083, LR = 4.1404612159329145e-06
Step 7000: Loss = N/A, LR = N/A


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step 7050: Loss = 0.0082, LR = 3.8128930817610063e-06
Step 7100: Loss = 0.0075, LR = 3.4853249475890986e-06
Step 7150: Loss = 0.009, LR = 3.1577568134171913e-06
Step 7200: Loss = 0.0084, LR = 2.830188679245283e-06
Step 7250: Loss = 0.0075, LR = 2.5026205450733754e-06
Step 7300: Loss = 0.008, LR = 2.1750524109014676e-06
Step 7350: Loss = 0.0082, LR = 1.8474842767295599e-06
Step 7400: Loss = 0.0078, LR = 1.519916142557652e-06
Step 7450: Loss = 0.0072, LR = 1.1923480083857442e-06
Step 7500: Loss = 0.0089, LR = 8.647798742138365e-07
Step 7500: Loss = N/A, LR = N/A


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step 7550: Loss = 0.0091, LR = 5.372117400419287e-07
Step 7600: Loss = 0.0092, LR = 2.09643605870021e-07
Step 7632: Loss = N/A, LR = N/A


In [10]:
pip install sacrebleu


Note: you may need to restart the kernel to use updated packages.


In [16]:
import torch

sample = eval_dataset[0]
input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
true_output = tokenizer.decode(sample["labels"], skip_special_tokens=True)


model.eval()
with torch.no_grad():
    pred_ids = model.generate(torch.tensor(sample["input_ids"]).unsqueeze(0).to(model.device))
    predicted_text = tokenizer.decode(pred_ids[0], skip_special_tokens=True)

print("Input Text:", input_text)
print("Expected Output:", true_output)
print("Model Prediction:", predicted_text)


Input Text: fix grammar: Things is improving in Algeria.
Expected Output: Things are improving in Algeria.
Model Prediction: Things are improving in Algeria.


In [18]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.5 MB/s eta 0:00:0000:01


In [36]:
import evaluate
import torch
from transformers import GenerationConfig
from tqdm import tqdm  
import random

eval_dataset_list = list(eval_dataset)

eval_dataset_sampled = random.sample(eval_dataset_list, 1000)

bleu = evaluate.load("sacrebleu")


generation_config = GenerationConfig(
    max_length=128,
    num_beams=5,
    early_stopping=True,
    decoder_start_token_id=tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
)


model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Evaluation function
def compute_metrics_on_dataset(model, tokenizer, eval_dataset):
    decoded_preds = []
    decoded_labels = []
    total_examples = len(eval_dataset)
    print(f"Total number of examples: {total_examples}")

    
    for i, example in enumerate(tqdm(eval_dataset, desc="Evaluating", total=total_examples)):  
        input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(device)
        label_ids = example["labels"]

    
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                **generation_config.to_dict()
            )

        
        pred_str = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        label_str = tokenizer.decode(label_ids, skip_special_tokens=True)

    
        decoded_preds.append(pred_str)
        decoded_labels.append(label_str)

    
    bleu_score = bleu.compute(predictions=decoded_preds, references=[[lbl] for lbl in decoded_labels])["score"]

    
    exact_matches = sum(1 for pred, gt in zip(decoded_preds, decoded_labels)
                        if pred.strip().lower() == gt.strip().lower())
    accuracy = (exact_matches / len(decoded_labels)) * 100

    return {"exact_match": accuracy, "bleu_score": bleu_score}

# Run evaluation
metrics = compute_metrics_on_dataset(model, tokenizer, eval_dataset_sampled)

print(f"Exact Match Accuracy: {metrics['exact_match']:.2f}%")
print(f"BLEU Score: {metrics['bleu_score']:.2f}")


Total number of examples: 1000


Evaluating: 100%|██████████| 1000/1000 [42:16<00:00,  2.54s/it]

Exact Match Accuracy: 24.90%
BLEU Score: 40.44
